In [1]:
import csv 
import torch
import torch.utils.data
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import argparse
from torch.optim import Adam
from torch.autograd import Variable
from RecurrentNet import RecurrentNet


filepath2 = "D:\\MLdata\\tracks.csv"
data2 = pd.read_csv(filepath2, index_col=0, header=[0,1])
y_col = data2.loc[:,( 'track', 'genre_top')].replace(to_replace=None, value=None, inplace=False, limit=None, regex=False, method='pad')
classes = set(y_col)
print(set(y_col))

{'Folk', 'Hip-Hop', 'Rock', 'Electronic', 'Old-Time / Historic', 'Easy Listening', 'Pop', 'Spoken', 'Blues', 'Soul-RnB', 'Country', 'Classical', 'Experimental', 'Instrumental', 'Jazz', 'International'}


In [2]:
import h5py
def read_hdf(file_path):
    return_dict = []
    with h5py.File(file_path, 'r') as hf:
        #print(list(hf.keys()))
        dataset = hf[file_path[9:-5]]
        #print(len(list(dataset.keys())))
        keys = list(dataset.keys())
        for i in keys:
            return_dict.append({
                "name":int(i.replace("-","/")[-10:-4]),
                "data":dataset[i][:]
            })
        return return_dict

In [3]:
from torch.optim import Adam
 
import argparse
import sys
import torch
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

class RecurrentNet(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_layers, dropout, netType):
        super(RecurrentNet, self).__init__()
        self.n_layers = num_layers
        self.hidden_size = hidden_size
        if netType == 'LSTM':
            self.rec = nn.LSTM(input_size, hidden_size, num_layers, dropout = dropout, batch_first = True)
        if netType == 'GRU':
            self.rec = nn.GRU(input_size, hidden_size, num_layers, dropout = dropout, batch_first = True)
        self.fc = nn.Linear(hidden_size, num_classes)
       
        
    def forward(self, x):
        out, _ = self.rec(x)
        out = out[:,-1, :]
        out = self.fc(out)
        return(out)


In [4]:
import random

x_data = []


#print(type(data))
n_outputs = 0
for k in classes:
    try:
        x_data.extend(read_hdf("D:/MLdata/" + k + ".hdf5"))
        
        n_outputs += 1
    except:
        pass
random.shuffle(x_data)

shape1 = len(x_data[0]['data'])

shape2 = len(x_data[0]['data'][0])


In [5]:
y_data = []
x_values = []
for i in x_data:
    y_data.append(y_col[i['name']])
    #x_values.append(i['data'].flatten())
    x_values.append((i['data']).flatten())
print(y_data[300])
#print(x_values[300][-5:-1])
#print(x_values[350][-5:-1])
#print(x_values[10][-5:-1])
#print(x_values[1000][-5:-1])
#print(len(x_values))
#for k in range(20):
#    for j in range(20):
#        print(len(x_values[k][j]))
#print(len(x_values[0][0]))

Pop


In [6]:
print(len(x_values))
print(x_values[0])
print(len(x_values[0]))

7994
[-37.227165 -35.425175 -53.35134  ... -64.781525 -67.23306  -64.57233 ]
82688


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler


label_encoder = LabelEncoder()
#x_col = x_data.to_numpy()
#print(len(y_col))
#print(len(x_col))
#print(len(x_col[0]))
#classes = set(y_col)
#print(len(classes))

#flatten x to normalize then unflatten
min_max_scaler = MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x_values)
#print(x_scaled[1])
x_values = np.reshape(x_scaled.astype(np.float32), (len(x_values),shape1,shape2))
#print("first element ox x")
#print(x_values[0])
counter = 0
x_transposed = []
for i in x_values:
    j = np.transpose(i)
    x_transposed.append(j)

print(len(x_transposed))

print(len(x_transposed[0]))

print(len(x_transposed[0][0]))
y_data = label_encoder.fit_transform(np.ravel(y_data))
y_data = y_data.reshape(-1, 1)
#print("possible values:")
#print(set(y_data))
#print(y_data)
encoder = OneHotEncoder(drop=None, sparse=False)
#print("possible values:")
#print(set(y_data))
# transform data
onehot_y = encoder.fit_transform(y_data)
#print(onehot_y)
#print(y_data)
X_train, X_test, y_train, y_test = train_test_split(x_transposed , onehot_y,  test_size=0.2)
#print(X_train[3])
#print(y_train[3])
X_train, X_test, y_train, y_test = torch.tensor(X_train), torch.tensor(X_test), torch.tensor(y_train).type(torch.LongTensor), torch.tensor(y_test).type(torch.LongTensor)

7994
646
128


C:\Users\tgraf\AppData\Local\Temp\ipykernel_20840\1303450908.py:48: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:204.)
  X_train, X_test, y_train, y_test = torch.tensor(X_train), torch.tensor(X_test), torch.tensor(y_train).type(torch.LongTensor), torch.tensor(y_test).type(torch.LongTensor)


In [8]:
#print(len(X_train))
#print(X_train[0])
#print(len(X_train[0]))
#print(len(X_train[0][0]))

In [9]:
from torch.autograd import Variable
import matplotlib.pyplot as plt



def testAccuracy(model):
    
    model.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for data in test_loader:
            audio, labels = data
            # run the model on the test set to predict labels
           # print("inputs")
            #print(audio[0])
            #print(len(audio))
            #print(len(audio[0]))
            outputs = model(audio)
            #print("outputs")
            #print(outputs[0])
            #print(len(outputs))
           # print(len(outputs[0]))
            # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            labels = torch.max(labels, 1)[1]
            #print("labels")
            #print(labels)
            #print("predicted")
            #print(predicted)
            accuracy += (predicted == labels).sum().item()
    
    accuracy = (100 * accuracy / total)
    return(accuracy)



def train(model, train_loader, num_epochs, lr, gamma):
    
    best_accuracy = 0.0
    accuracies = []
    optimizer = torch.optim.AdamW(net.parameters(),lr=lr,weight_decay=0.0001)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma)
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        running_acc = 0.0

        for i, (audio, labels) in enumerate(train_loader, 0):
            # zero the parameter gradients
            optimizer.zero_grad()
            
            outputs = model(audio)
        
            # compute the loss based on model output and real labels
            
            loss_fn = nn.CrossEntropyLoss()
            #print(loss_fn(outputs[0], torch.argmax(labels[0])))
            
            loss = loss_fn(outputs, torch.max(labels, 1)[1])
            #print(loss)
            # backpropagate the loss
            loss.backward()
            # adjust parameters based on the calculated gradients
            optimizer.step()
            running_loss += loss.item()     # extract the loss value
            
        accuracy = testAccuracy(model)
        
        accuracies.append(accuracy)
        if epoch % 10 == 0:
            scheduler.step()
            #print('epoch:', epoch+1,' test accuracy: %d %%' % (accuracy))
            #print(loss)
    plt.ylim([0, 100])    
    plt.plot(list(range(1,max_epochs+1)), accuracies)
    plt.xlabel("Number of epochs")
    plt.ylabel("Accuracy")
    plt.show()

In [10]:
print(y_train)

tensor([[0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 1, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0]])


In [ ]:
num_input = X_train.shape[1] - 1

#print(X_train[0])

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader  = torch.utils.data.DataLoader(train_dataset,
                                 batch_size=train_dataset.__len__())

test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
test_loader  = torch.utils.data.DataLoader(test_dataset,
                                 batch_size=test_dataset.__len__())



max_epochs = 50
#net = RecurrentNet(input_size = 128, hidden_size = 20, num_classes = 8, num_layers = 2, dropout = 0.1, netType = 'LSTM')
#train(net, train_loader, max_epochs, 0.03, 0.98)
for netType in ['LSTM', 'GRU']:
    for n_layers in [2,3]:
        print('Net Type:', netType,' number of layers: %d ' % (n_layers))   
        net = RecurrentNet(input_size = 128, hidden_size = 20, num_classes = 8, num_layers = n_layers, dropout = 0.1, netType = netType)
        train(net, train_loader, max_epochs, 0.01, 0.99)


Net Type: LSTM  number of layers: 2 


In [ ]:
# try only using one in every 19 time frames
X_subset = []
X_averages = []
X_ensemble = []


counter = 0
for i in x_transposed:
    xi_sub = []
    xi_avg = []
    xi_ensemble = []
    for bingus in range(19):
        xi_ensemble.append([])
    
    
    
    for time_subset in range(34):
        running_avg = np.zeros(128)
        xi_sub.append(i[19 * time_subset])
        for j in range(19):
            running_avg += i[19 * time_subset + j]
            xi_ensemble[j].append(i[19 * time_subset + j])
        xi_avg.append(running_avg/19)
    X_subset.append(xi_sub)
    X_averages.append(xi_avg)
    X_ensemble.append(xi_ensemble)

#print("hi")
#print(len(X_subset))
#print(len(X_subset[0]))
#print(len(X_subset[0][0]))

X_train, X_test, y_train, y_test = train_test_split(X_subset , onehot_y,  test_size=0.2)

X_train, X_test, y_train, y_test = torch.tensor(X_train), torch.tensor(X_test), torch.tensor(y_train).type(torch.LongTensor), torch.tensor(y_test).type(torch.LongTensor)
        
num_input = X_train.shape[1] - 1

#print(X_train[0])

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader  = torch.utils.data.DataLoader(train_dataset,
                                 batch_size=train_dataset.__len__())

test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
test_loader  = torch.utils.data.DataLoader(test_dataset,
                                 batch_size=test_dataset.__len__())


max_epochs = 1000
      
#net = RecurrentNet(input_size = 128, hidden_size = 30, num_classes = 8, num_layers = 2, dropout = 0.1, netType = 'LSTM')
#train(net, train_loader, max_epochs, 0.01, 0.997)        
        

In [ ]:
        
# try averaging X_train, X_test every 19 timesteps to reduce complexity
print("averages:")
X_train, X_test, y_train, y_test = train_test_split(X_averages, onehot_y,  test_size=0.2)

X_train, X_test, y_train, y_test = torch.tensor(X_train), torch.tensor(X_test), torch.tensor(y_train).type(torch.LongTensor), torch.tensor(y_test).type(torch.LongTensor)
        
num_input = X_train.shape[1] - 1

#print(X_train[0])

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader  = torch.utils.data.DataLoader(train_dataset,
                                 batch_size=train_dataset.__len__())

test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
test_loader  = torch.utils.data.DataLoader(test_dataset,
                                 batch_size=test_dataset.__len__())


max_epochs = 500
      
net = RecurrentNet(input_size = 128, hidden_size = 30, num_classes = 8, num_layers = 2, dropout = 0.1, netType = 'LSTM')
net = net.double()
train(net, train_loader, max_epochs, 0.01, 0.997)

In [ ]:
#try ensemble learning 

max_epochs = 300
models = []
test_sets = []
for k in range(6):
    net = RecurrentNet(input_size = 128, hidden_size = 30, num_classes = 8, num_layers = 2, dropout = 0.1, netType = 'LSTM')
    models.append(net)
    ensemble_train = []
    for data in X_ensemble:
        ensemble_train.append(data[3*k])
    X_train, X_test, y_train, y_test = train_test_split(X_subset , onehot_y,  test_size=0.2, random_state=42)

    X_train, X_test, y_train, y_test = torch.tensor(X_train), torch.tensor(X_test), torch.tensor(y_train).type(torch.LongTensor), torch.tensor(y_test).type(torch.LongTensor)
        
    num_input = X_train.shape[1] - 1
    
    test_sets.append((X_test, y_test))

#print(X_train[0])

    train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
    train_loader  = torch.utils.data.DataLoader(train_dataset,
                                 batch_size=train_dataset.__len__())

    test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
    test_loader  = torch.utils.data.DataLoader(test_dataset,
                                 batch_size=test_dataset.__len__())
    
    train(net, train_loader, max_epochs, 0.01, 0.997)



In [ ]:
votes = []
label_list =[]
    
for k in range(6):
    model = models[k]
    X_test, y_test = test_sets[k]
    test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
    test_loader  = torch.utils.data.DataLoader(test_dataset,
                                 batch_size=test_dataset.__len__())
    
    
    
    counter = 0
    for data in test_loader:
        
            
        audio, labels = data
           
        outputs = model(audio)
        votes.append(outputs)
        if len(label_list) == 0:
            label_list.append(labels)
        counter += 1

accuracy = 0            
#print(votes)
#print(sum(votes))
#print(sum(votes).type())
#print(label_list)
_, predicted = torch.max(sum(votes), 1)
    
labels = torch.max(label_list[0],1)[1]
accuracy += (predicted == labels).sum().item()
    
accuracy = (100 * accuracy / len(votes[0]))
print("Ensemble learning accuracy: %d %%" % (round(accuracy)))

    

In [ ]:
print(len(X_averages))
print(len(X_averages[0]))
print(len(X_averages[0][0]))

In [ ]:
print(len(X_ensemble))
print(len(X_ensemble[0]))
print(len(X_ensemble[0][0]))
print(len(X_ensemble[0][0][0]))